# Chapter 04-02: 데이터 증강 (Data Augmentation)

## 학습 목표
- Keras 내장 증강 레이어의 종류와 파라미터를 이해한다
- 모델 내부 레이어 방식과 `.map()` 전처리 방식의 차이를 안다
- `tf.image` API로 커스텀 증강 함수를 구현한다
- 증강이 과적합을 줄이는 원리를 이해한다

## 목차
1. Keras 증강 레이어
2. 모델 내부 vs 전처리 함수
3. tf.image API 커스텀 증강

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# 한글 폰트 설정 (macOS)
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

print('TensorFlow 버전:', tf.__version__)

# CIFAR-10 데이터 로드 (컬러 이미지라 증강 효과가 잘 보임)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# float32로 변환 및 정규화: [0,255] → [0.0, 1.0]
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32')  / 255.0

CLASS_NAMES = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

print(f'훈련 shape: {x_train.shape}  테스트 shape: {x_test.shape}')

## 수학적 기초

**2D 회전 변환 행렬**
$$R(\theta) = \begin{bmatrix}\cos\theta & -\sin\theta \\ \sin\theta & \cos\theta\end{bmatrix}$$

이미지의 각 픽셀 좌표 $(x,y)$에 위 행렬을 곱하면 $\theta$만큼 회전된 좌표를 얻는다.

**뒤집기(Flip)**: 수평 뒤집기는 $x' = W - 1 - x$ (이미지 너비 $W$)로 표현되며, 행렬로는
$$F = \begin{bmatrix}-1 & 0 \\ 0 & 1\end{bmatrix}$$
에 해당한다. 데이터 증강은 이런 기하학적 변환으로 훈련 데이터의 다양성을 인위적으로 늘려 모델의 일반화 성능을 높인다.

## 1. Keras 증강 레이어

In [ ]:
# -----------------------------------------------------------
# Keras 내장 증강 레이어 모음
# 각 레이어는 훈련(training=True) 시에만 확률적으로 적용된다
# 추론(training=False) 시에는 원본을 그대로 통과시킨다
# -----------------------------------------------------------

augmentation_layers = tf.keras.Sequential([
    # 좌우/상하 랜덤 뒤집기
    tf.keras.layers.RandomFlip('horizontal'),

    # ±20% 범위에서 랜덤 회전 (radian 단위: 0.2 ≈ 11.5°)
    tf.keras.layers.RandomRotation(factor=0.2),

    # 80%~120% 크기로 랜덤 줌 (음수: 줌아웃, 양수: 줌인)
    tf.keras.layers.RandomZoom(height_factor=0.2, width_factor=0.2),

    # 대비 랜덤 조정 (factor: 변화 비율)
    tf.keras.layers.RandomContrast(factor=0.2),

    # 밝기 랜덤 조정
    tf.keras.layers.RandomBrightness(factor=0.2),
], name='augmentation')

print('증강 레이어 구성:')
augmentation_layers.summary()

In [ ]:
# -----------------------------------------------------------
# 원본 이미지와 증강 결과 비교 시각화
# 같은 이미지를 9번 증강하여 다양성 확인
# -----------------------------------------------------------

sample_image = x_train[0]  # (32, 32, 3)
sample_batch = tf.expand_dims(sample_image, 0)  # (1, 32, 32, 3) — 배치 차원 추가

fig, axes = plt.subplots(3, 3, figsize=(9, 9))
fig.suptitle(f'원본: {CLASS_NAMES[y_train[0][0]]} — 동일 이미지 증강 9회', fontsize=14)

for i, ax in enumerate(axes.flat):
    if i == 0:
        # 왼쪽 상단: 원본
        ax.imshow(sample_image)
        ax.set_title('원본', fontsize=10)
    else:
        # training=True 로 호출해야 랜덤 증강이 적용된다
        augmented = augmentation_layers(sample_batch, training=True)
        ax.imshow(tf.clip_by_value(augmented[0], 0.0, 1.0))
        ax.set_title(f'증강 #{i}', fontsize=10)
    ax.axis('off')

plt.tight_layout()
plt.show()

## 2. 모델 내부 vs 전처리 함수

**방법 1 — 모델 내부 레이어 (추천)**: 증강이 훈련 시에만 자동 적용, GPU 병렬 처리

**방법 2 — .map() 전처리 함수**: 증강을 CPU에서 pipeline 단계에 적용

In [ ]:
# -----------------------------------------------------------
# 방법 1: 모델 내부 증강 레이어 (Functional API)
# 장점:
#   - model.predict() 시 training=False가 자동으로 설정되어
#     증강 레이어가 비활성화됨 (별도 처리 불필요)
#   - GPU에서 배치 단위로 빠르게 처리
# -----------------------------------------------------------

def build_model_with_augmentation(input_shape=(32, 32, 3), num_classes=10):
    inputs = tf.keras.Input(shape=input_shape)

    # --- 증강 블록 (훈련 시에만 활성) ---
    x = tf.keras.layers.RandomFlip('horizontal')(inputs)
    x = tf.keras.layers.RandomRotation(0.1)(x)
    x = tf.keras.layers.RandomZoom(0.1)(x)

    # --- 특징 추출 블록 ---
    x = tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.MaxPooling2D()(x)
    x = tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)

    # --- 분류기 ---
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    return tf.keras.Model(inputs, outputs, name='cnn_with_aug')

model_v1 = build_model_with_augmentation()
model_v1.summary()
print('\n[방법 1] 증강 레이어가 모델 첫 번째 블록에 포함되어 있음')

In [ ]:
# -----------------------------------------------------------
# 방법 2: .map() 파이프라인에서 증강 적용
# 장점:
#   - 증강 로직을 데이터 파이프라인과 분리하여 관리 가능
#   - CPU 다중 스레드(num_parallel_calls)로 병렬 처리
# 단점:
#   - 추론 시 별도의 파이프라인(증강 없는 버전)이 필요함
# -----------------------------------------------------------

AUTOTUNE = tf.data.AUTOTUNE

def augment_pipeline(image, label):
    """파이프라인용 증강 함수 — .map()에서 호출된다."""
    # 좌우 뒤집기: 50% 확률
    image = tf.image.random_flip_left_right(image)
    # 밝기 조정: ±0.2
    image = tf.image.random_brightness(image, max_delta=0.2)
    # 대비 조정: 0.8 ~ 1.2 배
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    # 범위 클리핑 (증강 후 [0,1] 범위를 벗어날 수 있음)
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, label

# 훈련용: 증강 포함
train_ds_aug = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(10_000, seed=42)
    .map(augment_pipeline, num_parallel_calls=AUTOTUNE)  # CPU 병렬 증강
    .batch(32)
    .prefetch(AUTOTUNE)
)

# 검증용: 증강 없음
val_ds = (
    tf.data.Dataset.from_tensor_slices((x_test, y_test))
    .batch(32)
    .prefetch(AUTOTUNE)
)

print('[방법 2] 파이프라인 구성 완료')
print('훈련 dataset spec:', train_ds_aug.element_spec)
print('검증 dataset spec:', val_ds.element_spec)

## 3. tf.image API 커스텀 증강

In [ ]:
# -----------------------------------------------------------
# tf.image API: 저수준 이미지 변환 함수들
# Keras 레이어보다 세밀한 제어가 필요할 때 사용한다
# -----------------------------------------------------------

sample = x_train[7]  # 테스트용 이미지

transformations = {
    '원본': sample,

    # 좌우 뒤집기
    'random_flip_left_right': tf.image.random_flip_left_right(sample),

    # 상하 뒤집기
    'random_flip_up_down': tf.image.random_flip_up_down(sample),

    # 밝기 조정: max_delta 범위에서 균일 분포로 샘플링
    'random_brightness': tf.image.random_brightness(sample, max_delta=0.4),

    # 대비 조정: [lower, upper] 범위에서 대비 배율 샘플링
    'random_contrast': tf.image.random_contrast(sample, lower=0.5, upper=1.5),

    # 채도 조정 (RGB 이미지 전용)
    'random_saturation': tf.image.random_saturation(sample, lower=0.5, upper=1.5),

    # 랜덤 크롭: 원본에서 24x24 영역을 잘라낸 후 다시 리사이즈
    'random_crop+resize': tf.image.resize(
        tf.image.random_crop(sample, size=(24, 24, 3)),
        size=(32, 32)
    ),

    # 색조 조정 (Hue)
    'random_hue': tf.image.random_hue(sample, max_delta=0.1),
}

# 시각화
n = len(transformations)
fig, axes = plt.subplots(2, 4, figsize=(14, 7))
fig.suptitle('tf.image API 커스텀 증강 갤러리', fontsize=14)

for ax, (name, img) in zip(axes.flat, transformations.items()):
    ax.imshow(tf.clip_by_value(img, 0.0, 1.0))
    ax.set_title(name, fontsize=9)
    ax.axis('off')

# 남은 칸 숨기기
for ax in axes.flat[n:]:
    ax.set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# -----------------------------------------------------------
# CutOut / Random Erasing 구현 — tf.image에 내장되지 않아
# 직접 구현해야 하는 고급 증강 기법
#
# 이미지의 무작위 사각형 영역을 0(검정)으로 가리면
# 모델이 특정 픽셀 위치에 과도하게 의존하지 않도록 강제한다
# -----------------------------------------------------------

@tf.function
def cutout(image, mask_size=8):
    """
    image: (H, W, C) float32 텐서
    mask_size: 가릴 정사각형 한 변의 길이 (픽셀)
    """
    h, w = tf.shape(image)[0], tf.shape(image)[1]

    # 마스크 중심 좌표를 이미지 범위 내에서 랜덤 선택
    cy = tf.random.uniform((), 0, h, dtype=tf.int32)
    cx = tf.random.uniform((), 0, w, dtype=tf.int32)

    # 마스크 영역 좌표 (이미지 경계 클리핑)
    y1 = tf.maximum(0, cy - mask_size // 2)
    y2 = tf.minimum(h, cy + mask_size // 2)
    x1 = tf.maximum(0, cx - mask_size // 2)
    x2 = tf.minimum(w, cx + mask_size // 2)

    # 패딩 마스크 생성: 해당 영역만 0, 나머지는 1
    # tf.ones → slices → 0으로 채우기
    mask_row = tf.zeros([y2 - y1, x2 - x1, tf.shape(image)[2]])
    mask_full = tf.ones_like(image)
    mask_full = tf.tensor_scatter_nd_update(
        mask_full,
        tf.stack(
            tf.meshgrid(
                tf.range(y1, y2),
                tf.range(x1, x2),
                tf.range(tf.shape(image)[2]),
                indexing='ij'
            ), axis=-1
        ),
        tf.zeros([(y2-y1)*(x2-x1)*tf.shape(image)[2]])
    )
    return image * tf.cast(mask_full, image.dtype)

# 간단한 버전 (numpy 방식 — eager mode에서만 동작)
def cutout_simple(image, mask_size=10):
    img = image.numpy().copy()
    h, w = img.shape[:2]
    cy, cx = np.random.randint(0, h), np.random.randint(0, w)
    y1, y2 = max(0, cy - mask_size//2), min(h, cy + mask_size//2)
    x1, x2 = max(0, cx - mask_size//2), min(w, cx + mask_size//2)
    img[y1:y2, x1:x2] = 0.0
    return img

# CutOut 시각화
fig, axes = plt.subplots(1, 5, figsize=(14, 3))
axes[0].imshow(sample)
axes[0].set_title('원본')
for i in range(1, 5):
    axes[i].imshow(cutout_simple(tf.constant(sample), mask_size=10))
    axes[i].set_title(f'CutOut #{i}')
for ax in axes:
    ax.axis('off')
plt.suptitle('CutOut 증강 (mask_size=10)')
plt.tight_layout()
plt.show()

## 정리

| 증강 방법 | 적용 대상 | 효과 |
|-----------|-----------|------|
| `RandomFlip` | 자연 이미지 | 뷰 방향 불변성 학습 |
| `RandomRotation` | 자연 이미지, 의료 이미지 | 회전 불변성 학습 |
| `RandomZoom` | 다양한 크기 객체 | 스케일 불변성 학습 |
| `RandomBrightness` / `RandomContrast` | 조명 변화가 많은 환경 | 조명 불변성 학습 |
| `RandomCrop` | 고해상도 이미지 | 위치 불변성 학습 |
| `CutOut` | 가림 현상이 있는 객체 | 부분 가림 내성 학습 |

**주의**: 증강은 훈련 데이터에만 적용한다. 검증/테스트 데이터에는 적용하지 않는다.

**다음**: 03_tfrecord_format.ipynb — TFRecord 포맷